In [5]:
import random
import numpy as np
from operator import attrgetter

def main():
    n_gene   = 100   # The number of genes.
    n_ind    = 300   # The number of individuals in a population.
    CXPB     = 0.5   # The probability of crossover.
    MUTPB    = 0.2   # The probability of individdual mutation.
    MUTINDPB = 0.05  # The probability of gene mutation.
    NGEN     = 40    # The number of generation loop.

    random.seed(64)
    # --- Step1 : Create initial generation.
    ##popはcreate_ind(n_gene)のリスト(長さrange(n_gene)分の0~1のどっちかの数のリスト)
    pop = create_pop(n_ind, n_gene)
    #pop関数をeval_funcに置き換えたリスト
    set_fitness(evalOneMax, pop)
    
    best_ind = max(pop, key=attrgetter("fitness"))
    
    
    # --- Generation loop.
    print("Generation loop start.")
    print("Generation: 0. Best fitness: " + str(best_ind.fitness))
    for g in range(NGEN):
        
        # --- Step2 : Selection.
        offspring = selTournament(pop, n_ind, tournsize=3)
        
        # --- Step3 : Crossover.
        crossover = []
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                child1, child2 = cxTwoPointCopy(child1, child2)
                child1.fitness = None
                child2.fitness = None
            crossover.append(child1)
            crossover.append(child2)

        offspring = crossover[:]
        
        # --- Step4 : Mutation.
        mutant = []
        for mut in offspring:
            if random.random() < MUTPB:
                mut = mutFlipBit(mut, indpb=MUTINDPB)
                mut.fitness = None
            mutant.append(mut)

        offspring = mutant[:]
        
        # --- Update next population.
        pop = offspring[:]
        set_fitness(evalOneMax, pop)
        
        # --- Print best fitness in the population.
        best_ind = max(pop, key=attrgetter("fitness"))
        print("Generation: " + str(g+1) + ". Best fitness: " + str(best_ind.fitness))
    
    print("Generation loop ended. The best individual: ")
    print(best_ind)
        
class Individual(np.ndarray):
    """Container of a individual."""
    fitness = None
    def __new__(cls, a):
        return np.asarray(a).view(cls)

def create_ind(n_gene):
    """Create a individual."""
    #create_ind(n_gene)は長さrange(n_gene)分の0~1のどっちかの数のリスト
    return Individual([random.randint(0, 1) for i in range(n_gene)])



#popを作製
def create_pop(n_ind, n_gene):
    """Create a population."""
    pop = []
    for i in range(n_ind):
        #ind = create_ind(n_gene)は一つ前に作ったdef文　変数はn_gene
        ind = create_ind(n_gene)
        
        #popはcreate_ind(n_gene)のリスト(長さrange(n_gene)分の0~1のどっちかの数のリスト)
        pop.append(ind)
    return pop

def set_fitness(eval_func, pop):
    """Set fitnesses of each individual in a population."""
    
    #zipは複数のリストでfor文を回すときに使う
    #map関数でpopをeval_funcに変換
    #.fitness は不明　
    for i, fit in zip(range(len(pop)), map(eval_func, pop)):
        pop[i].fitness = fit
      
        
def evalOneMax(ind):
    """Objective function."""
    #create_ind(n_gene)は長さrange(n_gene)分の0~1のどっちかの数のリストの合計
    #評価関数
    return sum(ind)

def selTournament(pop, n_ind, tournsize):
    """Selection function."""
    chosen = []    
    for i in range(n_ind):
        #popはcreate_ind(n_gene)のリスト(長さrange(n_gene)分の0~1のどっちかの数のリスト)
        #popからランダムにトーナメントサイズ分選ぶ
        aspirants = [random.choice(pop) for j in range(tournsize)]
        #トーナメントに選んだpopから最も優れた個体をリストに格納
        chosen.append(max(aspirants, key=attrgetter("fitness")))
        
    return chosen

def cxTwoPointCopy(ind1, ind2):
    """Crossover function."""
    size = len(ind1)
    tmp1 = ind1.copy()
    tmp2 = ind2.copy()
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size-1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    tmp1[cxpoint1:cxpoint2], tmp2[cxpoint1:cxpoint2] = tmp2[cxpoint1:cxpoint2].copy(), tmp1[cxpoint1:cxpoint2].copy()
    return tmp1, tmp2

def mutFlipBit(ind, indpb):
    """Mutation function."""
    tmp = ind.copy()
    #ind の中身を一定の確率で変える
    for i in range(len(ind)):
        if random.random() < indpb:
            tmp[i] = type(ind[i])(not ind[i])
    return tmp

if __name__ == "__main__":
    main()

42
56
45
55
50
51
50
58
50
58
48
53
56
49
48
50
59
56
49
55
46
57
54
55
51
50
48
46
48
50
49
54
48
53
58
59
55
54
49
61
47
52
43
53
64
56
52
58
46
50
41
55
55
46
47
50
48
51
42
50
54
40
47
45
62
50
40
46
46
48
55
47
56
54
42
48
35
43
53
55
46
50
53
45
53
46
55
34
49
51
49
45
46
44
44
44
52
48
53
48
54
48
48
50
64
56
47
36
49
51
58
56
46
55
49
47
46
39
46
48
40
55
62
55
49
42
50
47
53
54
54
63
48
59
47
43
47
50
49
52
49
45
44
54
58
47
52
52
52
44
51
56
42
54
55
46
50
57
49
59
51
63
54
54
47
51
56
48
52
58
52
55
44
43
45
56
52
51
52
53
51
53
45
44
48
50
57
49
46
57
57
49
46
53
43
52
43
51
50
51
51
48
42
51
52
53
55
44
55
49
43
57
52
49
50
54
59
48
38
54
53
49
59
54
45
49
52
56
55
50
44
59
47
45
49
48
44
48
48
64
41
45
39
46
47
51
59
54
54
52
56
46
48
56
50
49
45
52
51
53
55
49
53
51
48
48
47
52
62
53
48
51
52
49
45
62
59
60
47
51
53
42
57
48
58
54
46
51
55
53
53
51
49
47
56
61
45
47
41
53
Generation loop start.
Generation: 0. Best fitness: 64
60
55
57
55
53
57
51
53
56
48
50
60
57
48
54


75
76
70
74
75
73
74
74
72
73
76
76
70
75
77
70
70
74
72
75
74
75
72
75
76
78
73
76
70
76
76
80
77
75
75
75
72
74
74
72
75
67
74
72
77
79
77
74
76
74
74
73
76
74
74
74
72
72
79
73
73
75
73
75
76
69
72
72
74
72
73
79
72
74
80
73
74
71
78
76
74
74
72
74
72
82
64
72
74
74
76
69
74
72
74
76
77
74
75
73
73
72
73
73
73
75
76
75
74
74
76
76
68
75
73
74
75
76
73
75
64
78
70
74
73
75
72
74
73
77
71
72
77
75
78
75
73
73
75
74
75
76
76
73
74
74
75
74
69
76
71
68
75
Generation: 10. Best fitness: 82
80
75
75
78
75
74
77
77
73
77
78
80
72
78
76
79
77
74
79
76
77
71
66
74
82
77
79
73
73
77
75
73
75
80
76
74
77
77
77
71
74
76
78
75
76
77
73
78
76
76
74
77
79
78
79
80
77
76
75
77
77
77
76
75
75
80
76
77
75
77
76
76
65
78
69
77
74
79
78
75
79
75
75
76
77
73
74
70
78
67
75
72
73
79
75
75
75
77
75
76
76
77
77
76
76
71
74
73
74
76
79
74
79
75
77
75
78
75
71
76
80
74
74
75
73
70
76
75
76
77
76
79
79
76
74
77
77
76
77
75
77
76
76
75
75
76
70
78
68
77
74
76
76
74
76
75
71
79
75
72
78
71
72
77
76
74
74
71
73
7

87
85
88
87
86
87
87
86
85
84
87
88
87
86
88
88
87
88
89
86
87
86
88
86
87
88
89
86
87
87
88
87
86
88
87
88
84
84
88
87
80
84
86
79
88
88
88
87
Generation: 20. Best fitness: 91
88
90
89
89
88
89
87
89
88
85
88
86
88
88
85
90
88
88
88
84
88
82
90
87
88
84
90
88
89
89
86
89
89
83
89
87
86
86
89
88
89
88
88
87
88
88
80
90
88
88
78
87
86
89
87
89
82
87
83
83
88
88
90
88
88
88
86
85
87
89
86
89
88
88
87
88
89
87
88
86
90
89
88
89
89
88
87
81
79
89
83
87
87
88
91
88
87
87
87
88
88
85
88
89
90
90
89
87
89
83
87
88
86
89
88
88
88
86
89
88
88
83
89
84
88
88
88
84
88
86
88
88
89
89
87
88
88
88
87
88
86
88
87
88
89
88
85
89
89
89
88
83
89
88
89
88
89
88
87
88
89
83
88
88
86
88
89
90
81
88
87
88
89
90
87
88
84
84
89
88
85
78
88
89
86
88
87
89
90
88
91
88
90
79
90
86
81
78
87
89
80
87
87
84
89
88
83
88
89
88
88
88
88
90
85
88
88
85
90
88
89
89
88
87
88
88
85
88
89
87
87
88
88
88
87
85
89
89
87
89
87
89
89
88
84
87
90
87
88
91
88
88
84
88
86
86
87
88
88
87
87
88
91
85
87
87
88
87
83
87
82
88
81
88
8

96
95
97
95
94
93
96
96
96
96
93
94
93
94
93
95
96
96
96
92
96
94
96
93
95
89
96
96
96
94
94
94
95
95
96
94
93
95
94
94
89
97
93
96
95
92
95
95
93
96
95
97
93
91
95
92
93
95
93
95
95
95
84
96
88
96
90
96
96
96
96
95
92
89
89
97
95
94
97
97
93
96
87
95
96
95
93
96
94
91
96
93
93
96
97
90
94
97
96
94
94
95
94
96
96
95
96
93
92
96
92
95
96
95
95
92
94
89
96
94
97
94
94
92
94
89
87
95
96
94
96
93
96
96
96
97
94
94
97
97
95
95
96
94
96
95
95
94
96
95
95
96
95
90
95
96
96
91
94
97
97
92
93
96
94
96
97
95
95
96
95
96
95
86
94
92
Generation: 30. Best fitness: 97
91
95
96
95
96
95
96
95
97
95
96
97
95
91
93
92
96
96
96
91
93
96
95
96
95
96
97
96
97
97
97
95
96
95
94
95
95
91
97
95
96
96
97
96
96
95
96
95
92
97
93
96
90
97
98
95
95
96
96
96
96
98
96
86
97
96
95
98
96
96
95
96
97
96
94
98
95
96
97
97
96
97
91
85
96
96
93
95
96
96
95
95
96
90
92
97
96
90
97
96
97
95
96
95
93
95
96
90
94
91
95
95
95
97
93
96
96
97
96
95
94
92
94
95
90
94
96
94
96
93
90
97
96
96
94
97
96
96
97
96
93
95
96
90
95
96
9